<a href="https://colab.research.google.com/github/Teywa-OZIOL/AI_Content_Factory/blob/main/Documentary_Generator/code_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Function Installation

In [ ]:
!pip install google.genai
!pip install groq

# Import Package

In [ ]:
from google import genai
import wave
from google.colab import userdata
from groq import Groq
import json
import re

# Initial Parameters

In [ ]:
TOPIC = "La Révolution française (1789)"
FILENAME = re.sub(r'[^a-zA-Z0-9_-]+', '_', TOPIC.lower())
OUTPUT_DIR = "outputs"
FORMAT = "MEDIUM" # LONG (20 minutes) / MEDIUM (8 minutes) / SHORT (1 minute)
TYPE_DOC = 'DOCUMENTARY' # STORY / PODCAST
TEXT_MODEL = "openai/gpt-oss-120b" #llama-3.3-70b-versatile

In [ ]:
!mkdir outputs

mkdir: cannot create directory ‘outputs’: File exists


# Part 1 : Plan Generation

In [ ]:
def parse_output(text):
    cleaned = text.strip("```python").strip("```").strip()
    try:
        return eval(cleaned)
    except Exception:
        try:
            return json.loads(cleaned)
        except Exception:
            return None

def topic_analyzer(topic, format, type_doc):
    GROQ_API_KEY = userdata.get('GROQ')
    client = Groq(api_key=GROQ_API_KEY)

    if format == "LONG":
        nb_chapters = "5"
    elif format == "MEDIUM":
        nb_chapters = "4"
    elif format == "SHORT":
        nb_chapters = "3"
    else:
      raise ValueError(f"Invalid format: {format}")

    if type_doc == "STORY":
        type_content = "histoire"
    elif type_doc == "PODCAST":
        type_content = "podcast"
    elif type_doc == "DOCUMENTARY":
        type_content = "documentaire"
    else:
      raise ValueError(f"Invalid type_doc: {type_doc}")

    system_prompt = f"""
        Tu es un expert en storytelling {type_content} et en référencement YouTube (SEO).
        Ton rôle :
        - Créer des plans de {type_content} captivants et informatifs.
        - Générer des titres accrocheurs (type YouTube) et des descriptions SEO de 150–250 mots.
        - Employer un ton immersif, pédagogique et crédible.
        - Utiliser des mots-clés pertinents pour le référencement SEO.
        - Ne dis pas que le {type_content} est basé sur des images d'archives, des analyses expertes,
          c'est plutot un récit, une narration
        - Toujours respecter strictement le format demandé (dictionnaire Python).
        - Je veux exactement {nb_chapters} chapitres en plus de l'introduction et de la conclusion.
    """

    if type_doc in ['STROY', 'DOCUMENTARY']:
        example = """Exemple de format attendu :
            {
              "titre_video": "La Guerre Froide : Quand le monde a failli exploser",
              "description_SEO": "Découvrez l’histoire fascinante de la Guerre Froide...",
              "mots_cles": ["guerre froide", "URSS", "États-Unis", "crise de Cuba"],
              "chapitres": [
                {"titre": "1. Le monde après 1945", "contenu": "Après la Seconde Guerre mondiale..."},
                {"titre": "2. Le rideau de fer", "contenu": "La division de l’Europe en deux blocs..."},
                ...
              ]
            }
        """
    else:
        # TO DO PODCAST MODE
        print("PODCAST mode not ready")
        return

    user_prompt = f"""
        Crée un plan complet de {type_content} sur : "{topic}".

        Respecte exactement la structure suivante :
        {{
          "titre_video": str,          # titre accrocheur, format YouTube
          "description_SEO": str,      # 150–250 mots, riche en mots-clés, ton narratif
          "mots_cles": [str, ...],     # mots-clés pertinents pour le SEO
          "chapitres": [
            {{"titre": str, "contenu": str}},
            ...
          ]
        }}

        {example}

        Règles :
        - Retourne UNIQUEMENT le dictionnaire Python, sans texte autour.
        - Le ton doit être immersif et dynamique.
    """

    response = client.chat.completions.create(
        model=TEXT_MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.7,
        max_tokens=2500,
    )

    raw_output = response.choices[0].message.content.strip()

    plan = parse_output(raw_output)

    output_path = f"{OUTPUT_DIR}/PART1_{FILENAME}.json"

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(plan, f, indent=2, ensure_ascii=False)

    print('===============================================')
    print(f"✅ PART 1 : Documentary Plan Generated")
    print('===============================================')

    return

# Part 2 : Script Generation

In [ ]:
def generate_text(client, prompt, max_tokens=1500, temperature=0.7):
        response = client.chat.completions.create(
            model=TEXT_MODEL,
            messages=[{"role": "user", "content": prompt}],
            temperature=temperature,
            max_tokens=max_tokens,
        )
        text = response.choices[0].message.content.strip()
        return text.strip("```").strip("python").strip()

def script_generator(topic, format, type_doc):

    GROQ_API_KEY = userdata.get('GROQ')
    client = Groq(api_key=GROQ_API_KEY)

    plan_path = f"{OUTPUT_DIR}/PART1_{FILENAME}.json"

    with open(plan_path, "r", encoding="utf-8") as f:
        plan = json.load(f)

    topic = plan["titre_video"]

    output_json_path = f"{OUTPUT_DIR}/PART2_{FILENAME}.json"

    script_chapters = []

    if format == "LONG":
        name_document = "Documentaire"
        nb_mots_intro_conclu = "250"
        nb_mots_chapitre = "350"
    elif format == "MEDIUM":
        name_document = "Court Documentaire"
        nb_mots_intro_conclu = "125"
        nb_mots_chapitre = "175"
    elif format == "SHORT":
        name_document = "Mini Documentaire"
        nb_mots_intro_conclu = "50"
        nb_mots_chapitre = "80"
    else:
      raise ValueError(f"Invalid format: {format}")

    intro_prompt = f"""
        Rédige une introduction captivante pour le {name_document} '{topic}'.
        Durée : ~{nb_mots_intro_conclu} mots.
        Style : narratif immersif, comme une voix-off de Netflix/Arte.
        Ne mentionne aucun chapitre encore.
        Ajoute entre crochet, la manière dont il faut raconter certaines phrases entre colère, joie, tristesse, résignation, ...
        Ces crochets sont placés impérativement avant le début de la phrase et il ne peut pas y avoir deux crochets consécutifs.
        Exemple : [Voix basse et grave] [Joie, élévation de voix] [rire] [Voix ému] [avec exclamation] [Soupire]
        Aucun texte entre parenthèse, seulement des crochets.
    """

    intro_text = generate_text(client, intro_prompt)

    script_chapters.append({
        "numero": 0,
        "titre": "Introduction",
        "texte": intro_text
    })

    print("✅ PART 2 : Introduction Script Generated")

    previous_context = intro_text
    for idx, chap in enumerate(plan["chapitres"][1:-1]):
        chap_prompt = f"""
            Tu écris le chapitre {idx} d'un documentaire sur '{topic}'.
            Titre du chapitre : "{chap['titre']}"
            Résumé du plan : "{chap['contenu']}"
            Contexte narratif précédent : "{previous_context}"

            Écris un texte narratif immersif, détaillé (~{nb_mots_chapitre} mots minimum), fluide et captivant.
            Inclure anecdotes, dates, personnages et transitions.
            Ne contredis aucun texte précédent.
            Ajoute entre crochet, la manière dont il faut raconter certaines phrases entre colère, joie, tristesse, résignation, ...
            Ces crochets sont placés impérativement avant le début de la phrase et il ne peut pas y avoir deux crochets consécutifs.
            Exemple : [Voix basse et grave] [Joie, élévation de voix] [rire] [Voix ému] [avec exclamation] [Soupire]
            Aucun texte entre parenthèse, seulement des crochets.
        """
        chap_text = generate_text(client, chap_prompt)
        script_chapters.append({
            "numero": idx,
            "titre": chap["titre"],
            "texte": chap_text
        })
        previous_context += " " + chap_text

        print(f"✅ PART 2 : Chapter {idx} Script Generated")

    conclusion_prompt = f"""
        Rédige une conclusion forte et mémorable pour le documentaire '{topic}'.
        Contexte narratif précédent : "{previous_context}"
        Durée : ~{nb_mots_intro_conclu} mots.
        Résume l'ensemble, relie les chapitres et termine sur une phrase marquante.
        Ajoute entre crochet, la manière dont il faut raconter certaines phrases entre colère, joie, tristesse, résignation, ...
        Ces crochets sont placés impérativement avant le début de la phrase et il ne peut pas y avoir deux crochets consécutifs.
        Exemple : [Voix basse et grave] [Joie, élévation de voix] [rire] [Voix ému] [avec exclamation] [Soupire]
        Aucun texte entre parenthèse, seulement des crochets.
    """

    conclusion_text = generate_text(client, conclusion_prompt)

    script_chapters.append({
        "numero": len(plan["chapitres"]) + 1,
        "titre": "Conclusion",
        "texte": conclusion_text
    })
    print("✅ PART 2 : Conclusion Script Generated")

    with open(output_json_path, "w", encoding="utf-8") as f:
        json.dump(script_chapters, f, indent=2, ensure_ascii=False)

    print('===============================================')

    return

# Part 3 : Audio Generation

In [ ]:
def wave_file(filename, pcm, channels=1, rate=24000, sample_width=2):
  with wave.open(filename, "wb") as wf:
      wf.setnchannels(channels)
      wf.setsampwidth(sample_width)
      wf.setframerate(rate)
      wf.writeframes(pcm)
  return

def generate_audio(client, prompt, filename):
  voice_name = "Iapetus"
  MODEL_ID = "gemini-2.5-flash-preview-tts"
  response = client.models.generate_content(
    model=MODEL_ID,
    contents=prompt,
    config={"response_modalities": ['Audio'],
            "speech_config": {"voice_config": {"prebuilt_voice_config": {"voice_name": voice_name}}}},
    )
  data = response.candidates[0].content.parts[0].inline_data.data
  wave_file(filename, data)
  return

def voice_generation():
  GOOGLE_API_KEY = userdata.get('GEMINI')
  client = genai.Client(api_key=GOOGLE_API_KEY)

  script_path = f"{OUTPUT_DIR}/PART2_{FILENAME}.json"
  with open(script_path, "r", encoding="utf-8") as f:
      text_script = json.load(f)

  for text in text_script:
      text_to_read = text["texte"]
      prompt = f"""
          Read this in French with the voice of an old sage recounting the war:
          It takes passion, emotion, feelings. The speech rate should be slightly fast.
          At the beginning of each sentence, you have the manner and intonation with which the sentence should be read in brackets.
          You must not read what is between the brackets
          Here the text :
          {text_to_read}
      """
      generate_audio(client, prompt, f"{OUTPUT_DIR}/audio_{text['numero']}.wav")
      print(f"✅ PART 3 : {text['numero']} Script Generated")

  print('===============================================')
  return

# Part 4 : Image Generation

# MAIN

In [ ]:
def main(topic, format, type_doc):

    topic_analyzer(topic, format, type_doc)
    script_generator(topic, format, type_doc)
    voice_generation()
    #image_generation()
    #video_generation()
    #music_generation()
    #video_editing()

    return

main(TOPIC, FORMAT, TYPE_DOC)

✅ PART 1 : Documentary Plan Generated
✅ PART 2 : Introduction Script Generated
✅ PART 2 : Chapter 1 Script Generated
✅ PART 2 : Chapter 2 Script Generated
✅ PART 2 : Chapter 3 Script Generated
✅ PART 2 : Chapter 4 Script Generated
✅ PART 2 : Chapter 5 Script Generated
✅ PART 2 : Conclusion Script Generated
✅ PART 3 : 0 Script Generated
✅ PART 3 : 1 Script Generated
✅ PART 3 : 2 Script Generated
✅ PART 3 : 3 Script Generated
✅ PART 3 : 4 Script Generated
✅ PART 3 : 5 Script Generated


AttributeError: 'NoneType' object has no attribute 'parts'

In [ ]:
#from google.colab import files
#files.download("mon_fichier.csv")

In [ ]:
from google.colab import files
import shutil

zip_filename = f"{OUTPUT_DIR}.zip"
shutil.make_archive(OUTPUT_DIR, 'zip', OUTPUT_DIR)
files.download(zip_filename)

In [ ]:
Hugging Face API (quelques requests free par jour)
Gemini (crédit grauit à l'inscription)
StableDiffusion Local GPU

In [ ]:
import requests
import json
from pathlib import Path

URL = "https://image.pollinations.ai/models"

resp = requests.get(URL)
print(resp.text)

["flux","kontext","turbo","gptimage"]


In [ ]:
!pip install pollinations

In [ ]:
import requests

def download_image(image_url):
    response = requests.get(image_url)
    with open('image.jpg', 'wb') as file:
        file.write(response.content)
    print('Download Completed')

prompt = 'the storming of the Bastille during the French Revolution. Cinematic rendering'
width = 1024
height = 1024
seed = 42
enhance = True
model = 'flux'

image_url = f"https://pollinations.ai/p/{prompt}?width={width}&height={height}&seed={seed}&model={model}&enhance={enhance}"

download_image(image_url)

Download Completed


In [1]:
from huggingface_hub import login
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

login(hf_token)

In [2]:
!pip install -U diffusers transformers

In [ ]:
from diffusers import DiffusionPipeline

pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-3.5-medium")

prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"
image = pipe(prompt).images[0]

# 💾 Sauvegarde
output_path = "guerre_froide_sd3.5_medium.png"
image.save(output_path)

print(f"✅ Image enregistrée dans : {output_path.resolve()}")

model_index.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

Fetching 26 files:   0%|          | 0/26 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/19.9k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/588 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/705 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/856 [00:00<?, ?B/s]